In [1]:
import requests
import json
import pandas as pd
import numpy as np

In [2]:
r = requests.post('http://10.15.5.164:8000/backtest', data={'start_date': '2018-01-01',
                                                            'end_date': '2019-08-01',
                                                            'freq': '10b',
                                                            'max_round': 800})

In [3]:
result = json.loads(r.text)

In [4]:
ret_df = pd.DataFrame(json.loads(result['ret_df']))

In [5]:
from pyecharts import options as opts
from example.commons import Collector
from pyecharts.charts import Line

plot = ret_df[['xgb_regress', 'net_xgb_regress']].cumsum()
v1 = list(plot.index)
v2 = list(plot.xgb_regress)
v3 = list(plot.net_xgb_regress)

line_chart = (
    Line()
    .add_xaxis(v1)
    .add_yaxis("xgb_regress", v2)
    .add_yaxis("net_xgb_regress", v3)
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        datazoom_opts=[opts.DataZoomOpts(pos_bottom="-1%")],
        title_opts=opts.TitleOpts(title='Fixed freq rebalanced: {0}'.format('2b')),
    )
)

line_chart.render_notebook()

In [6]:
print("年化收益: {0:.2f}".format(len(ret_df) * np.mean(ret_df['net_xgb_regress'])))
print("夏普比率: {0:.2f}".format(np.sqrt(126) * np.mean(ret_df['net_xgb_regress']/ np.std(ret_df['net_xgb_regress']))))


年化收益: -0.57
夏普比率: -2.06


In [1]:
import torch
z = torch.Tensor(4, 5)
print(z)

tensor([[ 1.5196e-38,  0.0000e+00,  2.8026e-44,  0.0000e+00,  1.5245e-37],
        [ 0.0000e+00, -9.5466e-39,  4.5595e-41, -2.5964e-31,  4.5595e-41],
        [-9.5235e-39,  4.5595e-41,  0.0000e+00,  0.0000e+00,  5.3254e-37],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]])
